# Machine Learning Pipeline with KubeDirector - Lab 1
## Authenticate to Kubernetes cluster deployed on HPE Ezmeral Container Platform and fetch the kubeconfig file for your tenant

### **Lab workflow:**

In this lab:

1. As tenant user, you will first establish a valid login session with HPE Ezmeral Container Platform (HPE ECP) using a REST API call.

2. You will then fetch the *kubeconfig* file for your tenant working context using a subsequent HPE ECP REST API.
>Note: A kubeconfig file is used to access the Kubernetes cluster for your tenant working context.


**Recommended workshop and blogs about HPE Ezmeral Container Platform REST API:**
- Workshop: [Intro to the HPE Ezmeral Container Platform REST API](https://hackshack.hpedev.io/workshops) 
- Blog: [HPE Ezmeral Container Platform REST API Part 1 - Authenticating](https://developer.hpe.com/blog/hpe-container-platform-rest-api-part-1-authenticating)
- Blog: [HPE Ezmeral Container Platform REST API Part 2 - Deploying containerized applications](https://developer.hpe.com/blog/hpe-container-platform-rest-api-part-2-deploying-containerized-applicati)


**Definitions:**

- *HPE Ezmeral Container Platform* is an enterprise-grade container platform designed to deploy both cloud-native and non-cloud-native applications whether they are on-premises, at the edge, in multiple public clouds, or in a hybrid model. This makes the HPE Ezmeral Container Platform ideal for helping enterprise customers accelerate their application development and deployment via **containers** on-demand through a self-service portal and a RESTful API that surfaces programmable access. To learn more about HPE Ezmeral Container Platform, visit the [HPE DEV portal](https://developer.hpe.com/platform/hpe-ezmeral-container-platform/home) and check out the blog articles.

- *tenant:* A tenant is a group of users created by the platform administrator. A tenant can represent, for example, an office location, a business unit, an organization, or a project. A tenant is allocated a quota of resources (CPU, GPU, memory, storage, and Kubernetes clusters resources) by the platform administrator. All the resources used by a tenant are not shared with other tenants. The tenant serves as a Kubernetes namespace.

- *Working tenant context:* A working tenant context establishes the user identity, its tenant name and tenant role (member or admin). Based on this context, tenant users are granted privileges and permissions to create and manage resources for their tenant on Kubernetes clusters managed by HPE ECP.

#### Initialize the environment:

Let's first define the environment variables according to your HPE Ezmeral Container Platform user account and tenant name, and the HPE Ezmeral Container Platform API system endpoint.

**Make sure the password is set to the password you received in the registraton e-mail**

In [1]:
#
# environment variables to be verified by the student
#
studentId="student{{ STDID }}" # your Jupyter Notebook student Identifier (i.e.: student<xx>)
username="student{{ STDID }}" # your HPE ECP tenant login credentials - username 
password="{{ PASSSTU }}" # your HPE ECP tenant login credentials - password - Please replace with password provided in your registration

studentId="student74"
username="student74"
password="stuDISCO2020"
#
# fixed environment variables setup by the HPE ECP lab administrator - Please DO NOT MODIFY!!
#
gateway_host="{{ HPEECPGWNAME }}"
controller_endpoint="${gateway_host}:8080"

gateway_host="hpecpgw1.hp.local"
controller_endpoint="${gateway_host}:8080"

tenantname="K8sMLTenant" #case sensitive

echo "your operation context is:" $username "on tenant" $tenantname 
echo "your REST API endpoint is:" $controller_endpoint

your operation context is: student74 on tenant K8sMLTenant
your REST API endpoint is: hpecpgw1.hp.local:8080


#### Authenticate as a tenant user in the specified tenant:

> Note:  When you see a [*] next to the next action it means your execution step is busy working within the notebook.

>Note: Notice that the session location token will remain valid for 1440 minutes (or 24 hours), after which time you will have to establish a new login session.

In [2]:
sessionlocation=$(curl -k -i -s --request POST "https://${controller_endpoint}/api/v2/session" \
--header 'Accept: application/json' \
--header 'Content-Type: application/json' \
--data-raw '{
"name": "'"$username"'",
"password": "'"$password"'",
"tenant_name": "'"$tenantname"'"
}' | grep Location | awk '{print $2}' | tr -d '\r') #we remove any cr that might exist
echo "This is your session location: " $sessionlocation
SessionId=$(echo $sessionlocation | cut -d'/' -f 5) # extract sessionId for later, for logout
echo "This is your session_Id:" $SessionId

This is your session location:  /api/v2/session/57b9657a-1b3b-496f-b986-612d36ff046b
This is your session_Id: 57b9657a-1b3b-496f-b986-612d36ff046b


#### Get the Kubeconfig file for your tenant working context:
The next step in deploying a containerized application in Kubernetes clusters managed by the HPE Ezmeral Container Platform is to get the kubeconfig file for your tenant working context. 

The HPE ECP REST API call below allows you to get the **kubeconfig file** used to access the Kubernetes cluster for your tenant user account based on your assigned role (tenant member) with the same result as if you had downloaded it from the HPE ECP UI.

In [3]:
mkdir -p ~/.kube
ls ~/.kube

cache  config


In [4]:
rm -f ~/.kube/config
curl -k -s --request GET "https://${controller_endpoint}/api/v2/k8skubeconfig" \
--header "X-BDS-SESSION: $sessionlocation" \
--header 'Accept: application/json' \
--header 'Content-Type: application/json' > ~/.kube/config

#### Check your working tenant context:

In [5]:
kubectl config view

apiVersion: v1
clusters:
- cluster:
    insecure-skip-tls-verify: true
    server: https://hpecpgw1.hp.local:9500
  name: k8scluster1
- cluster:
    insecure-skip-tls-verify: true
    server: https://hpecpgw1.hp.local:9501
  name: k8scluster2
contexts:
- context:
    cluster: k8scluster1
    namespace: k8shacktenant
    user: HPECP-student74
  name: k8scluster1-K8sHackTenant-student74
- context:
    cluster: k8scluster2
    namespace: k8smltenant
    user: HPECP-student74
  name: k8scluster2-K8sMLTenant-student74
current-context: k8scluster2-K8sMLTenant-student74
kind: Config
preferences: {}
users:
- name: HPECP-student74
  user:
    exec:
      apiVersion: client.authentication.k8s.io/v1beta1
      args:
      - hpecp
      - authenticate
      - hpecpgw1.hp.local:8080
      - --hpecp-user=student74
      - --hpecp-token=/api/v2/session/57b9657a-1b3b-496f-b986-612d36ff046b
      - --hpecp-token-expiry=1608229303
      - --force-reauth=false
      - --insecure-skip-tls-verify=true
    

> Notice the kubeconfig file includes the valid token (session location) for the current session you previously established.

In [6]:
kubectl config current-context

k8scluster2-K8sMLTenant-student74


You can now send Kubernetes API requests using a K8s API client such as **kubectl** to set up a Machine Learning (ML) pipeline using KubeDirector and train, register, and query your model.


**Let's see this in action!**
* [Lab2 - Deploy Local Jupyter Notebook Cluster](2-WKSHP-K8s-ML-Pipeline-Deploy-Local-Notebook.ipynb)